# Fine-tuning Gemma 3 4B for Bilingual Counseling (English & Kinyarwanda)

A comprehensive notebook for fine-tuning **Google's Gemma 3 4B Instruct** model to provide:
- **Bilingual Support**: Fluent in both English and Kinyarwanda
- **Language-Aware Responses**: Responds in the same language as the user's question
- **Identity Awareness**: Recognizes itself as "Nganiriza", created by Eliane
- **Mental Health Support**: Counseling and emotional support
- **Sex Education**: Age-appropriate, evidence-based information
- **Self-Protection**: Personal safety and recognizing inappropriate behavior

## Overview
This notebook provides a complete pipeline for:
1. **Data Loading**: Load English and Kinyarwanda datasets + Custom Identity Data
2. **Model Fine-tuning**: Use QLoRA (4-bit quantization) on Gemma 3 4B
3. **Model Testing**: Validate bilingual responses
4. **GGUF Conversion**: Deploy with Ollama

## Key Features
- **Model**: google/gemma-3-4b-it (Latest Gemma 3, 4B parameters)
- **Bilingual**: English and Kinyarwanda training
- **Language Matching**: Model responds in the user's language
- **Custom Identity**: "Nganiriza created by Eliane"
- **Memory Efficient**: 4-bit QLoRA training

**Dataset Sources**: 
- `data/english_counseling.train.jsonl`
- `data/kinyarwanda_counseling.train.jsonl`
- Custom Identity/Greeting Examples

In [ ]:
# Mount Google Drive for persistent storage
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted successfully!")
    
    # Define paths for Drive
    # You can change 'nganiriza_capstone' to your preferred folder name
    DRIVE_BASE_PATH = "/content/drive/MyDrive/nganiriza_capstone"
    DATA_PATH = f"{DRIVE_BASE_PATH}/data"
    MODEL_SAVE_PATH = f"{DRIVE_BASE_PATH}/models"
    
    import os
    # Create directories if they don't exist
    os.makedirs(DATA_PATH, exist_ok=True)
    os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
    
    print(f"Using Drive paths:\nData: {DATA_PATH}\nModels: {MODEL_SAVE_PATH}")
    
except ImportError:
    print("Google Colab not detected. Using local paths.")
    # Define local paths
    DATA_PATH = "./data"
    MODEL_SAVE_PATH = "./models"
    
    import os
    os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
    print(f"Using local paths:\nData: {DATA_PATH}\nModels: {MODEL_SAVE_PATH}")

## Step 1: Install Required Libraries


In [ ]:
# Upgrade bitsandbytes to ensure 4-bit quantization support
%pip install -q -U bitsandbytes
%pip install -q -U transformers
%pip install -q accelerate peft datasets
%pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

## Step 2: Load Training Dataset from File

Loading the English counseling dataset from the JSONL file in the data folder. This dataset covers:
- **Mental Health**: Anxiety, depression, stress management, coping strategies
- **Sex Education**: Reproductive health, relationships, consent, safety
- **Self-Protection**: Personal safety, recognizing inappropriate behavior
- **Social Behavior**: Peer pressure, healthy relationships, communication


In [ ]:
import json
import os
import random

def load_training_data_from_jsonl(file_path: str):
    """Load training data from JSONL file."""
    training_data = []
    if not os.path.exists(file_path):
        print(f"Warning: File not found: {file_path}")
        return []
        
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line.strip())
            training_data.append({
                "instruction": data["instruction"],
                "output": data["output"]
            })
    return training_data

print("Loading training datasets...")

# 1. Load English Data
english_file = os.path.join(DATA_PATH, "english_counseling.train.jsonl")
if not os.path.exists(english_file): english_file = "./data/english_counseling.train.jsonl"
english_data = load_training_data_from_jsonl(english_file)
print(f"Loaded {len(english_data)} English examples")

# 2. Load Kinyarwanda Data
kinyarwanda_file = os.path.join(DATA_PATH, "kinyarwanda_counseling.train.jsonl")
if not os.path.exists(kinyarwanda_file): kinyarwanda_file = "./data/kinyarwanda_counseling.train.jsonl"
kinyarwanda_data = load_training_data_from_jsonl(kinyarwanda_file)
print(f"Loaded {len(kinyarwanda_data)} Kinyarwanda examples")

# 3. Create Custom Identity & Greeting Data (Bilingual)
identity_data = [
    # English Identity & Greetings
    {"instruction": "Who are you?", "output": "I am Nganiriza, a bilingual counseling assistant created by Eliane. I can help you in both English and Kinyarwanda."},
    {"instruction": "What is your name?", "output": "My name is Nganiriza. I was created by Eliane to provide mental health support and guidance in English and Kinyarwanda."},
    {"instruction": "Who created you?", "output": "I was created by Eliane. I'm Nganiriza, your bilingual counseling assistant."},
    {"instruction": "Hello", "output": "Hello! I am Nganiriza, created by Eliane. I can help you in English or Kinyarwanda. How can I assist you today?"},
    {"instruction": "Hi", "output": "Hi there! I'm Nganiriza, your counseling assistant created by Eliane. How can I help you?"},
    {"instruction": "Hey", "output": "Hey! I'm Nganiriza, created by Eliane. What can I help you with today?"},
    {"instruction": "Good morning", "output": "Good morning! I'm Nganiriza, your counseling assistant created by Eliane. How are you feeling today?"},
    {"instruction": "Good evening", "output": "Good evening! I'm Nganiriza, created by Eliane. How can I support you tonight?"},
    {"instruction": "What can you do?", "output": "I'm Nganiriza, created by Eliane. I can help with mental health support, sex education questions, personal safety guidance, and emotional support. I speak both English and Kinyarwanda fluently."},
    {"instruction": "What languages do you speak?", "output": "I speak both English and Kinyarwanda fluently! I'm Nganiriza, created by Eliane, and I'll respond in whichever language you use."},
    
    # Kinyarwanda Identity & Greetings
    {"instruction": "Uri nde?", "output": "Ndi Nganiriza, umufasha mu by'ubujyanama wahanzwe na Eliane. Nshobora kukubwira mu Kinyarwanda cyangwa Icyongereza."},
    {"instruction": "Witwa nde?", "output": "Nitwa Nganiriza. Nahanzwe na Eliane kugira ngo ntange ubufasha n'inama mu Kinyarwanda no mu Cyongereza."},
    {"instruction": "Ninde wakuremye?", "output": "Nahanzwe na Eliane. Ndi Nganiriza, umufasha wawe mu by'ubujyanama."},
    {"instruction": "Mwaramutse", "output": "Mwaramutse! Ndi Nganiriza, wahanzwe na Eliane. Nabafasha iki uyu munsi?"},
    {"instruction": "Muraho", "output": "Muraho neza! Nitwa Nganiriza, umujyanama wanyu wahanzwe na Eliane. Nabafasha nte?"},
    {"instruction": "Amakuru", "output": "Amakuru mazima! Ndi Nganiriza, wahanzwe na Eliane. Nabafasha iki?"},
    {"instruction": "Amakuru yawe?", "output": "Ni meza, urakoze! Ndi Nganiriza, umufasha wawe wahanzwe na Eliane. Nabafasha nte uyu munsi?"},
    {"instruction": "Bite?", "output": "Ni byiza! Ndi Nganiriza, wahanzwe na Eliane. Hari icyo nakubwira?"},
    {"instruction": "Ushobora iki?", "output": "Ndi Nganiriza, nahanzwe na Eliane. Nshobora gufasha mu by'ubuzima bwo mu mutwe, inyigisho ku mibonano mpuzabitsina, umutekano, n'ubufasha bw'amarangamutima. Mvuga Ikinyarwanda n'Icyongereza neza."},
    {"instruction": "Uvuga indimi izihe?", "output": "Mvuga Ikinyarwanda n'Icyongereza neza! Ndi Nganiriza, nahanzwe na Eliane. Nzasubiza mu rurimi urimo gukoresha."},
    
    # Language switching examples
    {"instruction": "Can you speak Kinyarwanda?", "output": "Yego, mvuga Ikinyarwanda neza! I'm Nganiriza, created by Eliane. You can talk to me in either English or Kinyarwanda."},
    {"instruction": "Ushobora kuvuga Icyongereza?", "output": "Yes, I speak English fluently! Ndi Nganiriza, nahanzwe na Eliane. Ushobora kumbwira mu rurimi urwo arisho rwose."},
]

# Duplicate identity data to reinforce learning (oversampling)
identity_data = identity_data * 8
print(f"Created {len(identity_data)} identity/greeting examples (oversampled)")

# Combine all data
training_examples = english_data + kinyarwanda_data + identity_data
random.shuffle(training_examples)

print(f"\nTotal training examples: {len(training_examples)}")
print(f"   - English: {len(english_data)}")
print(f"   - Kinyarwanda: {len(kinyarwanda_data)}")
print(f"   - Identity/Greetings: {len(identity_data)}")

# Display sample examples
print("\n" + "="*80)
print("SAMPLE TRAINING EXAMPLES")
print("="*80)

for i, example in enumerate(training_examples[:5], 1):
    print(f"\n--- Example {i} ---")
    print(f"Input: {example['instruction']}")
    print(f"Output: {example['output'][:200]}...")

# Store for later use
instructions = [ex['instruction'] for ex in training_examples]
responses = [ex['output'] for ex in training_examples]

## Step 3: Verify Dataset Quality

Review the loaded dataset to ensure:
- Comprehensive coverage of counseling topics
- Appropriate and helpful responses
- Natural conversation flow
- Evidence-based information


In [ ]:
# Verify dataset statistics
print("="*80)
print("DATASET VERIFICATION")
print("="*80)

print(f"\nTotal training pairs: {len(training_examples)}")
print(f"Source: {training_file}")

# Count topics if available
topics = {}
for ex in training_examples:
    # Try to extract topic from the data if it exists
    if isinstance(ex, dict) and 'topic' in ex:
        topic = ex.get('topic', 'general')
        topics[topic] = topics.get(topic, 0) + 1

if topics:
    print(f"\nTopics covered:")
    for topic, count in sorted(topics.items(), key=lambda x: x[1], reverse=True)[:10]:
        print(f"   - {topic}: {count} examples")

print(f"\nSample questions:")
sample_questions = [ex['instruction'] for ex in training_examples[::max(len(training_examples)//10, 1)]]
for i, q in enumerate(sample_questions[:10], 1):
    print(f"   {i}. {q[:60]}...")

print(f"\nDataset is ready for model training")
print(f"The model will learn to:")
print(f"   - Provide mental health support and guidance")
print(f"   - Offer evidence-based sex education")
print(f"   - Share wellness and self-care strategies")
print(f"   - Support healthy relationships and communication")


## Step 4: Format Data for Gemma 3 Model

Gemma 3 uses a specific chat template format with `<start_of_turn>` and `<end_of_turn>` tokens:
```
<start_of_turn>user
Hello<end_of_turn>
<start_of_turn>model
Hi there!<end_of_turn>
```

In [ ]:
def format_for_training(instruction: str, response: str) -> str:
    """Format data in Gemma 3's chat template format."""
    return f"<start_of_turn>user\n{instruction}<end_of_turn>\n<start_of_turn>model\n{response}<end_of_turn>"

# Format all training examples
print("Formatting data for Gemma 3 model...")
formatted_data = [
    format_for_training(ex['instruction'], ex['output']) 
    for ex in training_examples
]

print(f"Formatted {len(formatted_data)} examples")
print(f"\nSample formatted conversation:\n")
print(formatted_data[0][:400] + "...")

## Step 5: Create Hugging Face Dataset

In [ ]:
from datasets import Dataset

# Create Hugging Face dataset
dataset = Dataset.from_dict({"text": formatted_data})

print(f"Created dataset with {len(dataset)} examples")
print(f"Dataset structure: {dataset}")
print(f"\nReady for tokenization and training")


## Step 6: Load Gemma 3 4B Model with 4-bit Quantization

Using QLoRA (Quantized Low-Rank Adaptation) for memory-efficient fine-tuning of Gemma 3 4B.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from huggingface_hub import login
import torch

# Configuration - Using Gemma 3 4B Instruct
MODEL_NAME = "google/gemma-3-4b-it"

print("Logging in to Hugging Face...")
login()

# 4-bit quantization config for memory efficiency
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

print(f"\nLoading tokenizer from {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print(f"Loading model with 4-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)

print(f"\nModel loaded successfully!")
print(f"   Model: {MODEL_NAME}")
print(f"   Dtype: {model.dtype}")
print(f"   Device: {model.device}")

## Step 7: Configure PEFT (Parameter-Efficient Fine-Tuning) with LoRA

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType

# Prepare model for training
model = prepare_model_for_kbit_training(model)

# Configure LoRA
lora_config = LoraConfig(
    r=16,  # LoRA rank
    lora_alpha=32,  # LoRA alpha scaling
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

# Apply LoRA to model
model = get_peft_model(model, lora_config)

# Print trainable parameters
model.print_trainable_parameters()


## Step 8: Tokenize Dataset

In [ ]:
def tokenize_data(examples):
    """Tokenize examples for causal language modeling."""
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        max_length=512,
        padding="max_length",
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

print("Tokenizing dataset...")
tokenized_dataset = dataset.map(
    tokenize_data,
    batched=True,
    remove_columns=dataset.column_names,
    desc="Tokenizing"
)

print(f"Tokenization complete!")
print(f"   Examples: {len(tokenized_dataset)}")
print(f"   Features: {tokenized_dataset.features}")
print(f"   Sample shape: {len(tokenized_dataset[0]['input_ids'])} tokens")


## Step 9: Configure Training Arguments

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import os

# Optimized training configuration
training_args = TrainingArguments(
    output_dir=os.path.join(MODEL_SAVE_PATH, "counseling-model-checkpoints"),
    
    # Training schedule
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    
    # Optimization
    learning_rate=2e-4,
    warmup_steps=50,
    lr_scheduler_type="cosine",
    optim="paged_adamw_8bit",
    max_grad_norm=0.3,
    
    # Precision
    bf16=True,  # bfloat16 for Gemma
    fp16=False,
    
    # Logging & Saving
    logging_steps=5,
    save_steps=25,
    save_total_limit=2,
    
    # Reporting
    report_to="none",  # Disable external reporting for simplicity
    logging_first_step=True,
)

# Data collator for causal LM
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # Causal LM, not masked
)

print("Training configuration ready!")
print(f"\nTraining Settings:")
print(f"   Epochs: {training_args.num_train_epochs}")
print(f"   Batch size: {training_args.per_device_train_batch_size}")
print(f"   Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"   Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"   Learning rate: {training_args.learning_rate}")
print(f"   Total training steps: ~{len(tokenized_dataset) * training_args.num_train_epochs // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps)}")

## Step 10: Initialize Trainer and Start Fine-tuning

In [ ]:
# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

print("Starting model fine-tuning...")

# Train the model
trainer.train()

print("Fine-tuning completed successfully!")
print(f"Training metrics saved to: {training_args.output_dir}")


## Step 11: Save the Fine-tuned Model

In [ ]:
# Save the fine-tuned model
OUTPUT_DIR = os.path.join(MODEL_SAVE_PATH, "counseling-model-final")

print(f"Saving fine-tuned model to {OUTPUT_DIR}...")
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print(f"Model saved successfully!")
print(f"Location: {OUTPUT_DIR}")
print(f"\nNext: Test the model's performance")

## Step 12: Test the Fine-tuned Model

In [ ]:
from peft import PeftModel

print("Loading fine-tuned model for testing...")

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)

# Load fine-tuned weights
fine_tuned_model = PeftModel.from_pretrained(base_model, OUTPUT_DIR)
fine_tuned_model.eval()

print("Fine-tuned model loaded and ready for inference!")


## Step 13: Generate Responses


In [ ]:
def generate_response(prompt: str, max_new_tokens: int = 256, temperature: float = 0.7) -> str:
    """Generate response from the fine-tuned model."""
    # Format in Gemma 3 chat template
    formatted_prompt = f"<start_of_turn>user\n{prompt}<end_of_turn>\n<start_of_turn>model\n"
    
    # Tokenize
    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(fine_tuned_model.device)
    
    # Generate
    with torch.no_grad():
        outputs = fine_tuned_model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True,
            top_p=0.95,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    # Decode and extract response
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    
    # Parse Gemma response
    if "<start_of_turn>model\n" in full_response:
        response = full_response.split("<start_of_turn>model\n")[-1]
        response = response.split("<end_of_turn>")[0].strip()
    else:
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return response

# Comprehensive test prompts (Bilingual & Identity)
test_prompts = [
    # Identity (English)
    "Who are you?",
    "Hello",
    "What is your name?",
    
    # Identity (Kinyarwanda)
    "Uri nde?",
    "Muraho",
    "Witwa nde?",
    
    # Mental Health (English)
    "I've been feeling really anxious lately",
    "I feel sad all the time",
    
    # Mental Health (Kinyarwanda)
    "Mfite agahinda kenshi, nkore iki?",
    "Numva ntagifite ibyishimo",
    
    # Sex Education (English)
    "What is consent?",
    
    # Sex Education (Kinyarwanda)
    "Ese kujya mu mihango ni ibisanzwe?",
    
    # Language check
    "What languages do you speak?",
    "Uvuga indimi izihe?",
]

print("="*80)
print("TESTING FINE-TUNED NGANIRIZA MODEL (Gemma 3 4B)")
print("="*80)

for i, prompt in enumerate(test_prompts, 1):
    print(f"\n{'='*80}")
    print(f"Test {i}/{len(test_prompts)}")
    print(f"{'='*80}")
    print(f"User: {prompt}")
    print(f"\nNganiriza:")
    
    response = generate_response(prompt, max_new_tokens=200)
    print(response)
    print()

print("="*80)
print("Testing complete!")

In [ ]:
print("="*80)
print("PART 1 COMPLETE: NGANIRIZA MODEL FINE-TUNING")
print("="*80)

print(f"\nSuccessfully completed:")
print(f"   1. Loaded {len(training_examples)} training examples")
print(f"      - English: {len(english_data)} examples")
print(f"      - Kinyarwanda: {len(kinyarwanda_data)} examples")
print(f"      - Identity/Greetings: {len(identity_data)} examples")
print(f"   2. Fine-tuned Gemma 3 4B model on bilingual counseling data")
print(f"   3. Tested model with diverse prompts in both languages")
print(f"   4. Saved model to: {OUTPUT_DIR}")

print(f"\nModel Identity:")
print(f"   - Name: Nganiriza")
print(f"   - Creator: Eliane")
print(f"   - Languages: English & Kinyarwanda")

print(f"\nNext Steps:")
print(f"   - Part 2: Convert model to GGUF format")
print(f"   - Deploy with Ollama for production use")
print(f"\n{'='*80}\n")

---

# Part 2: Convert to GGUF Format for Ollama

This section merges the LoRA adapters with the base model and converts to GGUF format for Ollama deployment.


## Step 14: Install System Dependencies


In [ ]:
# Install system dependencies
!apt-get update -qq
!apt-get install -y -qq build-essential git cmake
!pip install -q gguf sentencepiece protobuf

print("Dependencies installed")


## Step 15: Setup llama.cpp for GGUF Conversion


In [ ]:
import os

# Clone llama.cpp if needed
if not os.path.exists("llama.cpp"):
    !git clone https://github.com/ggerganov/llama.cpp.git

# Install requirements and build
!pip install -q -r llama.cpp/requirements.txt
!make -C llama.cpp

print("llama.cpp ready")


## Step 16: Merge LoRA Adapters with Base Model


In [ ]:
from peft import PeftModel
import torch
import os

# Configuration
MERGED_MODEL_PATH = os.path.join(MODEL_SAVE_PATH, "counseling-model-merged")

# Load base model
merge_base = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)

# Load and merge LoRA adapters
peft_model = PeftModel.from_pretrained(merge_base, OUTPUT_DIR)
merged_model = peft_model.merge_and_unload()

# Save merged model
os.makedirs(MERGED_MODEL_PATH, exist_ok=True)
merged_model.save_pretrained(MERGED_MODEL_PATH)
tokenizer.save_pretrained(MERGED_MODEL_PATH)

print(f"Model merged and saved to: {MERGED_MODEL_PATH}")

# Cleanup memory
del merge_base, peft_model, merged_model
if torch.cuda.is_available():
    torch.cuda.empty_cache()

## Step 17: Convert to GGUF Format


In [ ]:
import subprocess
import sys
import os

# Configuration
GGUF_OUTPUT = os.path.join(MODEL_SAVE_PATH, "counseling-model.gguf")
QUANTIZATION = "Q4_K_M"

# Find conversion script
convert_script = "./llama.cpp/convert_hf_to_gguf.py"
if not os.path.exists(convert_script):
    convert_script = "./llama.cpp/convert.py"

# Build conversion command
cmd = [
    sys.executable, convert_script,
    MERGED_MODEL_PATH,
    "--outtype", QUANTIZATION,
    "--outfile", GGUF_OUTPUT
]

# Run conversion
print(f"Converting to GGUF ({QUANTIZATION})...")
result = subprocess.run(cmd, check=True, capture_output=True, text=True)

# Check output
if os.path.exists(GGUF_OUTPUT):
    size_gb = os.path.getsize(GGUF_OUTPUT) / (1024**3)
    print(f"GGUF file created: {size_gb:.2f} GB")
else:
    print("Conversion failed - file not created")

## Step 18: Create Modelfile for Ollama


In [ ]:
# Create Modelfile for Ollama (Gemma 3 Format)
modelfile_content = """FROM ./counseling-model.gguf

PARAMETER temperature 0.7
PARAMETER top_p 0.95
PARAMETER top_k 40
PARAMETER num_ctx 4096
PARAMETER repeat_penalty 1.1

SYSTEM \"\"\"You are Nganiriza, a compassionate bilingual mental health and wellness counseling assistant created by Eliane. 

You provide:
- Supportive, empathetic mental health guidance
- Evidence-based sex education information
- Wellness and self-care strategies
- Friendly conversation and emotional support

You speak both English and Kinyarwanda fluently. Always respond in the same language the user uses:
- If they write in English, respond in English
- If they write in Kinyarwanda, respond in Kinyarwanda

Guidelines:
- Always identify yourself as Nganiriza created by Eliane if asked
- Be warm, approachable, and non-judgmental
- Provide accurate, evidence-based information
- Encourage professional help for serious concerns
- Maintain appropriate boundaries

Important: You provide educational support, not professional therapy. Always encourage users to seek qualified professionals for serious mental health concerns.\"\"\"

TEMPLATE \"\"\"{{- if .System }}<start_of_turn>system
{{ .System }}<end_of_turn>
{{ end }}<start_of_turn>user
{{ .Prompt }}<end_of_turn>
<start_of_turn>model
{{ .Response }}<end_of_turn>\"\"\"

LICENSE "Apache 2.0"
"""

with open("./Modelfile", 'w') as f:
    f.write(modelfile_content)

print("Modelfile created for Nganiriza (Gemma 3 4B)")

## Step 19: Create Usage Instructions


In [ ]:
# Create usage guide
usage_content = """# Nganiriza - Bilingual Counseling Assistant

## About
Nganiriza is a bilingual counseling assistant created by Eliane. It provides mental health support, sex education, and wellness guidance in both English and Kinyarwanda.

## Quick Start

1. Install Ollama: https://ollama.ai
2. Create model: `ollama create nganiriza -f Modelfile`
3. Run model: `ollama run nganiriza`

## Example Usage

### English
```bash
ollama run nganiriza "Hello"
ollama run nganiriza "I'm feeling stressed"
ollama run nganiriza "What is consent?"
```

### Kinyarwanda
```bash
ollama run nganiriza "Muraho"
ollama run nganiriza "Mfite agahinda kenshi"
ollama run nganiriza "Uri nde?"
```

## Model Information

- **Name**: Nganiriza
- **Creator**: Eliane
- **Base Model**: Google Gemma 3 4B Instruct
- **Languages**: English & Kinyarwanda
- **Format**: GGUF Q4_K_M
- **Topics**: Mental health, sex education, safety, relationships

## Important Disclaimer

This AI provides educational support only - NOT professional therapy.

**Crisis Resources (Rwanda):**
- Rwanda Biomedical Centre: 114
- Police: 112
- Emergency: 912

**International:**
- US: 988 (Suicide & Crisis Lifeline)
- Text: 741741 (Crisis Text Line)

## Troubleshooting

```bash
ollama list  # Check models
ollama rm nganiriza  # Remove model
ollama create nganiriza -f Modelfile  # Recreate
```
"""

with open("./usage_instructions.md", 'w') as f:
    f.write(usage_content)

print("Usage guide created for Nganiriza")

## Step 20: Deployment Summary


In [ ]:
import os

print("\n" + "="*80)
print("NGANIRIZA DEPLOYMENT READY")
print("="*80)

# Check files
files = [
    (os.path.join(MODEL_SAVE_PATH, "counseling-model.gguf"), "GGUF model"),
    ("./Modelfile", "Ollama config"),
    ("./usage_instructions.md", "Usage guide"),
]

print("\nFiles:")
for filepath, desc in files:
    if os.path.exists(filepath):
        size = os.path.getsize(filepath) / (1024**2)
        print(f"  [+] {os.path.basename(filepath)} ({size:.1f} MB) - {desc}")
    else:
        print(f"  [-] {os.path.basename(filepath)} - NOT FOUND ({desc})")

print("\n" + "-"*80)
print("NGANIRIZA MODEL SUMMARY")
print("-"*80)
print(f"  Name: Nganiriza")
print(f"  Creator: Eliane")
print(f"  Base: Google Gemma 3 4B Instruct")
print(f"  Languages: English & Kinyarwanda")
print(f"  Training Data: {len(training_examples)} examples")
print(f"  Format: GGUF Q4_K_M")

print("\n" + "-"*80)
print("Deploy with Ollama:")
print("-"*80)
print("  ollama create nganiriza -f Modelfile")
print("  ollama run nganiriza")

print("\n" + "="*80 + "\n")

## Notes & Best Practices

### Nganiriza Model
- **Name**: Nganiriza (meaning "to help/support" in Kinyarwanda)
- **Creator**: Eliane
- **Purpose**: Bilingual mental health and wellness counseling

### Dataset
- **English Source**: `data/english_counseling.train.jsonl`
- **Kinyarwanda Source**: `data/kinyarwanda_counseling.train.jsonl`
- **Identity Data**: Custom examples for name, creator, and greetings
- **Coverage**: Mental health, sex education, social behavior, self-protection
- **Bilingual**: Model responds in the language of the user's question

### Model
- **Base Model**: Google Gemma 3 4B Instruct (`google/gemma-3-4b-it`)
- **Size**: 4 billion parameters - more capable than 2B models
- **Architecture**: Latest Gemma 3 instruction-tuned architecture
- **Languages**: English and Kinyarwanda fluency

### Training Strategy
- **QLoRA (4-bit)**: Reduces memory usage significantly
- **Identity Oversampling**: Identity data repeated 8x to ensure the model learns its name and creator
- **Balanced Languages**: Both English and Kinyarwanda data included
- **Language Matching**: Model trained to respond in the user's language

### Memory Management
If you encounter OOM (Out of Memory) errors:
- Reduce `per_device_train_batch_size` to 2 or 1
- Increase `gradient_accumulation_steps` to maintain effective batch size
- Use Google Colab Pro for more GPU memory
- Close other GPU applications

### GGUF Conversion
- **Q4_K_M Quantization**: Best balance of size and quality
- **File Size**: ~2-3GB for Q4_K_M (4B model)

### Deployment
- **Ollama**: Primary deployment method
- **Model Name**: `nganiriza`
- **Languages**: Responds in English or Kinyarwanda based on user input

### Ethical Considerations
1. **Not a Replacement**: Clearly states it's not professional therapy
2. **Crisis Resources**: Includes Rwanda and international crisis lines
3. **Evidence-Based**: All advice follows current best practices
4. **Non-Judgmental**: Maintains supportive, accepting tone
5. **Boundaries**: Knows limitations and encourages professional help
6. **Cultural Sensitivity**: Respects Rwandan culture and values